In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from itertools import combinations

In [2]:
zerodha = pd.read_csv('TELIA1.HE.csv')
lycaa = pd.read_csv('TSCO.L.csv')
tesla = pd.read_csv('TSLA.csv')

In [3]:
zerodha['stock_name'] = 'zerodha'
lycaa['stock_name'] = 'lycaa'
tesla['stock_name'] = 'tesla'

In [4]:
combined_df = pd.concat([zerodha, lycaa, tesla], ignore_index=True)

In [5]:
basket = (combined_df.groupby(['Date', 'stock_name'])['stock_name']
          .count().unstack().reset_index().fillna(0)
          .set_index('Date'))

In [6]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [7]:
basket_sets = basket.applymap(encode_units)

frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

In [8]:
print(frequent_itemsets)

    support                 itemsets
0  1.000000                  (lycaa)
1  0.448165                  (tesla)
2  0.454058                (zerodha)
3  0.448165           (tesla, lycaa)
4  0.454058         (zerodha, lycaa)
5  0.448165         (tesla, zerodha)
6  0.448165  (tesla, zerodha, lycaa)


In [9]:
combined_df.to_csv('combined_stock_data.csv', index=False)

In [10]:
print(rules)

         antecedents       consequents  antecedent support  \
0            (tesla)           (lycaa)            0.448165   
1            (lycaa)           (tesla)            1.000000   
2          (zerodha)           (lycaa)            0.454058   
3            (lycaa)         (zerodha)            1.000000   
4            (tesla)         (zerodha)            0.448165   
5          (zerodha)           (tesla)            0.454058   
6   (tesla, zerodha)           (lycaa)            0.448165   
7     (tesla, lycaa)         (zerodha)            0.448165   
8   (zerodha, lycaa)           (tesla)            0.454058   
9            (tesla)  (zerodha, lycaa)            0.448165   
10         (zerodha)    (tesla, lycaa)            0.454058   
11           (lycaa)  (tesla, zerodha)            1.000000   

    consequent support   support  confidence     lift  leverage  conviction  
0             1.000000  0.448165    1.000000  1.00000  0.000000         inf  
1             0.448165  0.448165    0

In [11]:
filtered_rules = rules[rules['lift'] > 1.2]

In [12]:
sorted_rules = filtered_rules.sort_values(by='lift', ascending=False)

In [13]:
print("Best combinations to buy:")
for index, row in sorted_rules.iterrows():
    items = ', '.join(row['antecedents']) + " => " + ', '.join(row['consequents'])
    lift = row['lift']
    print(f"Items: {items}, Lift: {lift}")

Best combinations to buy:
Items: tesla => zerodha, Lift: 2.2023598820059
Items: zerodha => tesla, Lift: 2.2023598820059
Items: tesla, lycaa => zerodha, Lift: 2.2023598820059
Items: zerodha, lycaa => tesla, Lift: 2.2023598820059
Items: tesla => zerodha, lycaa, Lift: 2.2023598820059
Items: zerodha => tesla, lycaa, Lift: 2.2023598820059
